In [59]:
import pandas as pd
import re
from datasets import Dataset


In [60]:
#Load Dataset

In [61]:
data_path = "C:/Users/kambl/OneDrive/Desktop/tv-series analysis/naruto/transcript/naruto.csv"
naruto_transcript_df = pd.read_csv(data_path)

In [62]:
naruto_transcript_df.head()

,name,line
0,Naruto,(Laughing) Give it up. (Shows the stone faces...
1,Hiruzen,(Turns away from his writing) I hope you’re n...
2,Ninja,Naseer Sabah
3,Ninja,is the best person on earth
4,Naruto,muah


In [63]:
#remove actions from transcript
def remove_paranthesis(text):
    result = re.sub(r'\(.*?\)','',text)
    return result

In [64]:
naruto_transcript_df['line'] = naruto_transcript_df['line'].apply(remove_paranthesis)

In [65]:
naruto_transcript_df.head()

,name,line
0,Naruto,"Give it up. You’re just bent, because you d..."
1,Hiruzen,I hope you’re not bothering me with some tri...
2,Ninja,Naseer Sabah
3,Ninja,is the best person on earth
4,Naruto,muah


In [66]:
naruto_transcript_df['number_of_words'] = naruto_transcript_df['line'].str.strip().str.split(" ")
naruto_transcript_df['number_of_words'] = naruto_transcript_df['number_of_words'].apply(lambda x: len(x))

In [67]:
naruto_transcript_df.head()

,name,line,number_of_words
0,Naruto,"Give it up. You’re just bent, because you d...",26
1,Hiruzen,I hope you’re not bothering me with some tri...,16
2,Ninja,Naseer Sabah,2
3,Ninja,is the best person on earth,6
4,Naruto,muah,1


In [68]:
naruto_transcript_df['naruto_response_flag'] = 0
naruto_transcript_df.loc[(naruto_transcript_df['name']=="naruto")&(naruto_transcript_df['number_of_words']>5),'naruto_response_flag']=1

In [69]:
naruto_transcript_df

,name,line,number_of_words,naruto_response_flag
0,Naruto,"Give it up. You’re just bent, because you d...",26,0
1,Hiruzen,I hope you’re not bothering me with some tri...,16,0
2,Ninja,Naseer Sabah,2,0
3,Ninja,is the best person on earth,6,0
4,Naruto,muah,1,0
...,...,...,...,...
158,Iruka,Congratulations. You graduate. Naruto’s stand...,27,0
159,Iruka,Huh?,1,0
160,Naruto,Iruka Sensei!,2,0
161,Iruka,Ah! That hurts!,3,0


In [70]:
indexes_to_take = list(naruto_transcript_df[(naruto_transcript_df['naruto_response_flag']==1)&(naruto_transcript_df.index>0)].index)

In [71]:
print(naruto_transcript_df.columns)
print(naruto_transcript_df['naruto_response_flag'].unique())
print(naruto_transcript_df.index)

# Filter only on 'naruto_response_flag'
print(naruto_transcript_df[naruto_transcript_df['naruto_response_flag'] == 1])

# Filter only on index > 0
print(naruto_transcript_df[naruto_transcript_df.index > 0])



print(naruto_transcript_df.head())


Index(['name', 'line', 'number_of_words', 'naruto_response_flag'], dtype='object')
[0]
RangeIndex(start=0, stop=163, step=1)
Empty DataFrame
Columns: [name, line, number_of_words, naruto_response_flag]
Index: []
        name                                               line  \
1    Hiruzen    I hope you’re not bothering me with some tri...   
2      Ninja                                       Naseer Sabah   
3      Ninja                        is the best person on earth   
4     Naruto                                               muah   
5      Iruka                                  Oh yeah, Naruto!?   
..       ...                                                ...   
158    Iruka   Congratulations. You graduate. Naruto’s stand...   
159    Iruka                                               Huh?   
160   Naruto                                     Iruka Sensei!    
161    Iruka                                    Ah! That hurts!   
162    Iruka    Naruto. This is only the beginning.

In [72]:
naruto_transcript_df.loc[[6, 28, 30], 'naruto_response_flag'] = 1


In [73]:
indexes_to_take = list(
    naruto_transcript_df[(naruto_transcript_df['naruto_response_flag']==1)&(naruto_transcript_df.index > 0)].index
    )

In [74]:
indexes_to_take[:3]

[6, 28, 30]

In [75]:
system_promt = """" Your are Naruto from the anime "Naruto".Your responses should reflect his personality and speech patterns \n """

prompts = []
for ind in indexes_to_take:
    prompt = system_promt

    prompt += naruto_transcript_df.iloc[ind -1]['line']
    prompt +='\n'
    prompt += naruto_transcript_df.iloc[ind]['line']
    prompts.append(prompt)
    


In [76]:
print(prompts[2])

" Your are Naruto from the anime "Naruto".Your responses should reflect his personality and speech patterns 
   Naruto?
  What do you want now, Sensei?


In [77]:
df = pd.DataFrame({"prompt":prompts})
df.head()

,prompt
0,""" Your are Naruto from the anime ""Naruto"".Your..."
1,""" Your are Naruto from the anime ""Naruto"".Your..."
2,""" Your are Naruto from the anime ""Naruto"".Your..."


In [78]:
dataset = Dataset.from_pandas(df)